In [16]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

data_list = []

csv_files = [f for f in os.listdir('data/')] 
for file in csv_files:
    file_path = os.path.join('data/', file)
    data_list.append(pd.read_csv(file_path))

df = pd.concat(data_list, ignore_index=True)

df[df['class'] == 'back-slouch'] #Grabs only a specific class.

    

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z32,v32,x33,y33,z33,v33,neck_tilt,shoulder_tilt,mouth_tilt,eye_tilt
0,back-slouch,0.509448,0.524600,-0.522874,0.999766,0.522115,0.491173,-0.499200,0.999721,0.528078,...,0.017744,0.025120,0.437206,1.868234,0.106335,0.011936,30.468780,112.674779,139.763642,142.459910
1,back-slouch,0.514575,0.521808,-0.545932,0.999773,0.526927,0.488849,-0.514747,0.999734,0.534232,...,0.158668,0.028173,0.440444,1.970423,0.146236,0.015577,30.317279,115.427690,128.360529,130.256377
2,back-slouch,0.515132,0.520802,-0.558694,0.999788,0.528085,0.488747,-0.520031,0.999754,0.535698,...,0.152009,0.026906,0.443615,2.081633,0.108856,0.015270,30.678226,113.655348,128.445598,130.600115
3,back-slouch,0.517845,0.519300,-0.536412,0.999783,0.531239,0.487632,-0.492078,0.999755,0.538949,...,0.182745,0.026613,0.453008,2.142330,0.090229,0.016016,30.861230,112.990364,131.319567,130.669615
4,back-slouch,0.519888,0.519069,-0.510512,0.999778,0.533118,0.487201,-0.465184,0.999753,0.540787,...,0.154047,0.024961,0.456965,2.190049,0.033289,0.015050,31.096813,111.697687,130.607744,130.688743
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1024,back-slouch,0.564399,0.303206,-0.855443,0.999966,0.592667,0.247874,-0.795125,0.999957,0.606145,...,-0.028262,0.010116,0.362117,2.893229,-0.074945,0.007097,48.929026,135.224992,145.411370,153.473523
1025,back-slouch,0.563819,0.303093,-0.846320,0.999967,0.591965,0.248326,-0.789353,0.999958,0.605964,...,-0.034409,0.010024,0.358491,2.883461,-0.064909,0.007008,49.253662,136.444429,147.468708,156.751476
1026,back-slouch,0.564684,0.306226,-0.886273,0.999970,0.593858,0.254182,-0.829716,0.999961,0.607586,...,0.006765,0.009985,0.350178,2.856667,-0.056730,0.007025,49.068984,139.824470,149.196942,160.430054
1027,back-slouch,0.567939,0.306806,-0.871285,0.999971,0.596494,0.255345,-0.816093,0.999963,0.610170,...,-0.032956,0.009961,0.373099,2.862422,-0.056777,0.006941,49.472869,139.876650,151.231323,161.933108


In [17]:
X = df.drop('class', axis=1) #Features (no class col)
y = df['class'] #Target value (class col)


In [18]:
#Create train/ test split.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234)

# Train Classification Model

In [19]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [20]:
"""pipelines = {
    'lr':make_pipeline(StandardScaler(), LogisticRegression(max_iter=200)),
    'rc':make_pipeline(StandardScaler(), RidgeClassifier(max_iter=200)),
    'rf':make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gb':make_pipeline(StandardScaler(), GradientBoostingClassifier())
}

#Train every model (4) on the data.
fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train, y_train)
    fit_models[algo] = model"""

pipeline = make_pipeline(StandardScaler(), RandomForestClassifier())
model = pipeline.fit(X_train, y_train)


In [21]:
model.predict(X_test)

array(['forward-slouch', 'side-lean', 'back-slouch', ...,
       'forward-slouch', 'side-lean', 'back-slouch'], dtype=object)

# Evaluate & Serialise Models

In [9]:
from sklearn.metrics import accuracy_score
import pickle

In [22]:
"""for algo, model in fit_models.items():
    yhat = model.predict(X_test)
    print(algo, accuracy_score(y_test, yhat))"""

yhat = model.predict(X_test)
print(f"rf: {accuracy_score(y_test, yhat)}")

rf: 0.9944894651539709


In [23]:
"""# SAVE MODEL #
with open('pose_detection.pkl', 'wb') as f:
    pickle.dump(fit_models['lr'], f)"""

with open('pose_detection.pkl', 'wb') as f:
    pickle.dump(model, f)

# Make Detections w/ Model

In [12]:
# RELOAD INTO NOTEBOOK #
with open('pose_detection.pkl', 'rb') as f:
    model = pickle.load(f)

